In [5]:
#Fetch the transcript from the csv file Game_of_Thrones_Script.csv
#Format: Release Date,Season,Episode,Episode Title,Name,Sentence
#Collect all characters and their sentences in a dictionary for each episode
#Store the dictionary in a list for each season

import csv
import re
import pandas as pd

#Read the csv file
df = pd.read_csv('Game_of_Thrones_Script.csv')

#Create a list of all the seasons
seasons = df.Season.unique().tolist()

#Create a list of all the episodes
episodes = df.Episode.unique().tolist()

#Create a list of all the characters
characters = df.Name.unique().tolist()

#Create a list of all the sentences
sentences = df.Sentence.unique().tolist()

#Create a list of all the episode. Give each episode a name season_X_episode_Y, where X is the season number and Y is the episode number
episodes_list = []
for season in seasons:
    for episode in episodes:
        episodes_list.append('season_' + str(season) + '_episode_' + str(episode))


#Collect all characters and their sentences in a dictionary for each episode
#Store the dictionary in a list for each season
""" seasons_list = []

for season in seasons:
    season_dict = {}
    for episode in episodes:
        episode_dict = {}
        for character in characters:
            character_dict = {}
            for sentence in sentences:
                if sentence in df[(df['Season'] == season) & (df['Episode'] == episode) & (df['Name'] == character)]['Sentence'].values:
                    character_dict[sentence] = sentence
            episode_dict[character] = character_dict
        season_dict[episode] = episode_dict
    seasons_list.append(season_dict) """


#For each episode create a list of all the characters with the character name and csv line number. Do not include the sentences that are not spoken by any character
#Store the list in a dictionary for each season
seasons_list = []

for season in seasons:
    season_dict = {}
    for episode in episodes:
        episode_dict = {}
        for character in characters:
            interaction_list = []
            #put all interations in a list
            for sentence in sentences:
                if sentence in df[(df['Season'] == season) & (df['Episode'] == episode)]:
                    #Add the character name and csv line number to the list
                    interaction_list.append([character, df[(df['Season'] == season) & (df['Episode'] == episode) & (df['Name'] == character) & (df['Sentence'] == sentence)].index.values[0]])
                    
            episode_dict[character] = interaction_list
        season_dict[episode] = episode_dict
    seasons_list.append(season_dict)




AttributeError: 'DataFrame' object has no attribute 'Sentence'

In [1]:
import pandas as pd
from collections import defaultdict

# Load the CSV file into a DataFrame
df = pd.read_csv('Game_of_Thrones_Script.csv')

# Initialize a dictionary to hold interaction matrices for each episode
interaction_matrices = defaultdict(lambda: defaultdict(int))

# Iterate over the DataFrame
for i, row in df.iterrows():
    # Get the current character and episode
    current_character = row['Name']
    episode_key = (row['Season'], row['Episode'])

    # Find interactions by looking at the next 5 lines
    for j in range(i+1, min(i+6, len(df))):
        next_row = df.iloc[j]
        # Check if the next line is within the same episode
        if episode_key == (next_row['Season'], next_row['Episode']):
            next_character = next_row['Name']
            if current_character != next_character:
                # Increment the interaction count in both directions
                interaction_matrices[episode_key][(current_character, next_character)] += 1
                interaction_matrices[episode_key][(next_character, current_character)] += 1

# Consolidate the interaction counts so each pair is only counted once
consolidated_interactions = {}

for episode_key, interactions in interaction_matrices.items():
    consolidated_interactions[episode_key] = defaultdict(int)
    for (char_a, char_b), count in interactions.items():
        if (char_b, char_a) in consolidated_interactions[episode_key]:
            continue  # Skip if already counted in reverse order
        total_count = count + interactions.get((char_b, char_a), 0)
        consolidated_interactions[episode_key][(char_a, char_b)] = total_count



In [2]:
#For each season print the top 10 interactions in the matrix
for season in seasons:
    print('Season: ' + str(season))
    for episode in episodes:
        print('Episode: ' + str(episode))
        print(sorted(consolidated_interactions[(season, episode)].items(), key=lambda x: x[1], reverse=True)[:10])
        print('\n')

NameError: name 'seasons' is not defined